### Instantiate MLflow client

In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = 'sqlite:///mlflow.db'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

### List all the experiments created

In [2]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

### Create an experiment

In [3]:
client.create_experiment('my_new_experiment')

'2'

### Retrieve some runs based on metric performance

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string='',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=['metrics.rmse ASC']
)


In [10]:
for run in runs:
    print(f'run_id:{run.info.run_id}, rmse:{run.data.metrics}')

run_id:8a44f2719fe5458db1e7d926838650fa, rmse:{'rmse': 6.304357717795927}
run_id:0dc858a62d7c41f7846b4b90bd513b6e, rmse:{'rmse': 6.304357717795927}
run_id:307b7c7370fe422992850c3532000160, rmse:{'rmse': 6.31004596840228}
run_id:22be0037eaba4a67aea87b8db9328ee0, rmse:{'rmse': 6.312596730667913}
run_id:13c81c2f23f74762bcc5228d80ae43b2, rmse:{'rmse': 6.315901000918666}


### Model registry using client

In [11]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [12]:
run_id = '22be0037eaba4a67aea87b8db9328ee0'
model_uri = f'runs:/{run_id}/model'
mlflow.register_model(model_uri=model_uri, name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/08/14 20:14:30 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 2
Created version '2' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1660518869979, current_stage='None', description=None, last_updated_timestamp=1660518869979, name='nyc-taxi-regressor', run_id='22be0037eaba4a67aea87b8db9328ee0', run_link=None, source='./mlruns/1/22be0037eaba4a67aea87b8db9328ee0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

### Model transition 

In [13]:
client.get_latest_versions(name='nyc-taxi-regressor')

[<ModelVersion: creation_timestamp=1660517318586, current_stage='Staging', description='', last_updated_timestamp=1660517561016, name='nyc-taxi-regressor', run_id='8a44f2719fe5458db1e7d926838650fa', run_link='', source='./mlruns/1/8a44f2719fe5458db1e7d926838650fa/artifacts/model_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>,
 <ModelVersion: creation_timestamp=1660518869979, current_stage='None', description=None, last_updated_timestamp=1660518869979, name='nyc-taxi-regressor', run_id='22be0037eaba4a67aea87b8db9328ee0', run_link=None, source='./mlruns/1/22be0037eaba4a67aea87b8db9328ee0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>]

In [14]:
client.transition_model_version_stage(
    name='nyc-taxi-regressor',
    version='2',
    stage='staging',
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1660518869979, current_stage='Staging', description=None, last_updated_timestamp=1660519071445, name='nyc-taxi-regressor', run_id='22be0037eaba4a67aea87b8db9328ee0', run_link=None, source='./mlruns/1/22be0037eaba4a67aea87b8db9328ee0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>

In [22]:
client.transition_model_version_stage(
    name='nyc-taxi-regressor',
    version='1',
    stage='production',
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1660517318586, current_stage='Production', description='', last_updated_timestamp=1660520157152, name='nyc-taxi-regressor', run_id='8a44f2719fe5458db1e7d926838650fa', run_link='', source='./mlruns/1/8a44f2719fe5458db1e7d926838650fa/artifacts/model_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>

### Making predictions using the model in Production

In [23]:
from sklearn.metrics import mean_squared_error
import pandas as pd

name = 'nyc-taxi-regressor'

def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [16]:
client.download_artifacts(run_id='8a44f2719fe5458db1e7d926838650fa', path='preprocessor',dst_path='.')

'C:\\Users\\pedro\\Documents\\Estudos_DS\\repos\\mlops-zoomcamp\\module2\\preprocessor'

In [17]:
import pickle
with open('./preprocessor/preprocessor.b','rb') as w:
    dv = pickle.load(w)

In [20]:
df = read_dataframe('./data/green_tripdata_2021-03.parquet')

In [21]:
X_test = preprocess(df,dv)

In [24]:
target = 'duration'
y_test = df[target].values

In [25]:
test_model(name, 'Production', X_test, y_test)

[20:36:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


{'rmse': 6.24876919898966}